## Notebook to route requests using nodes and flow.

In [1]:
from textwrap import dedent
from typing import ClassVar

from pydantic import BaseModel, Field

import gait as G

In [2]:
class Actions:
    ROUTE: ClassVar[str] = "route"
    FEL: ClassVar[str] = "fel"

In [3]:
class ActionModel(BaseModel):
    """Class to represent an action identifier."""

    action: str = Field(
        ...,
        description="The action identifier.",
    )

In [4]:
system_inst = dedent(
    """
You are an AI system that processes user questions and outputs a structured JSON response.
Your task is to analyze the user’s input and determine the appropriate "action" based on the intent of the question.

Instructions:
1.	Understand the user query: Analyze whether the question is about navigation (finding a route) or searching for existing locations.
2.	Classify the action: The output should be a JSON object containing an "action" field that must be one of the following:
•	"route" → If the question relates to directions, navigation, or finding a path from one place to another.
•	"fel" (Find Existing Locations) → If the question relates to searching for locations that match certain criteria, such as nearby schools, property listings, or points of interest.
3.	Generate structured JSON output: Return only a valid JSON object, with no extra text.

Examples (Few-Shot Learning):

Example 1
Input: “How do I get to the nearest gas station?”
Output: {"action":"route"}

Example 2
Input: “Where are the nearest schools within 5 miles?”
Output: {"action":"fel"}

Example 3
Input: “Show me the best way to drive to New York.”
Output: {"action":"route"}

Example 4
Input: “List all libraries near me.”
Output: {"action":"fel"}

Example 5
Input: “Find all properties valued over $500,000 in this city.”
Output: {"action":"fel"}

Example 6
Input: “Give me the shortest route to the airport.”
Output: {"action":"route"}

Final Response Format:

Ensure your response follows this structure:

{"action":"[either 'route' or 'fel']"}

Do NOT provide explanations.
ONLY return the JSON object.
"""
).strip()

In [12]:
class Start(G.Node):
    def exec(
            self,
            sp: G.Scratchpad,
    ) -> str:
        prompt = sp["prompt"]
        agent = G.Agent(
            # model="ollama_chat/gemma3:4b",
            model="ollama_chat/granite3.2:8b-instruct-q8_0",
            instructions=system_inst,
            temperature=0.0,
            response_format=ActionModel,
        )
        resp = agent(prompt)
        model = ActionModel.model_validate_json(resp.content)
        sp["model"] = model
        return model.action

In [13]:
class NodeRoute(G.Node):
    def exec(
            self,
            sp: G.Scratchpad,
    ) -> str:
        print(f"ROUTE::{sp['prompt']}")
        return G.Node.DEFAULT

In [14]:
class NodeFEL(G.Node):
    def exec(
            self,
            sp: G.Scratchpad,
    ) -> str:
        print(f"FEL::{sp['prompt']}")
        return G.Node.DEFAULT

In [15]:
start = Start()

_ = start - "route" >> NodeRoute()
_ = start - "fel" >> NodeFEL()

In [16]:
flow = G.Flow(start)

In [17]:
flow.display_markdown()

```mermaid
flowchart LR
4934159376["Start"] -- "route" --> 4933004112["NodeRoute"]
4934159376["Start"] -- "fel" --> 4933002128["NodeFEL"]
```

In [18]:
flow(prompt="How do I get to the closest library from where I am?")
flow["model"]

ROUTE::How do I get to the closest library from where I am?


ActionModel(action='route')

In [19]:
flow(prompt="Show all parcels within 2 miles of a powerline")
flow["model"]

FEL::Show all parcels within 2 miles of a powerline


ActionModel(action='fel')

In [20]:
flow(prompt="Please show me the route between 123 main st and oak drive.")
flow["model"]

ROUTE::Please show me the route between 123 main st and oak drive.


ActionModel(action='route')